# Make a series of Chloropleth Maps showing average sentiment of covid tweets over time with slider
### input: average_tweet_sentiment csvs for each day of April
### output: plotly chloropleth map html

In [1]:
# importing libraries

from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.figure_factory as ff
import numpy as np
import pandas as pd
import plotly.express as px


Below we create a list of the paths to each csv to analyse

In [2]:
paths_list=[]
for i in range(1,31): # day 1 to 30 for the month of April
    if i>9:
        paths_list.append('average_sentiment_{}.csv'.format(i))
    else:
        paths_list.append('average_sentiment_0{}.csv'.format(i))

print(paths_list)

['average_sentiment_01.csv', 'average_sentiment_02.csv', 'average_sentiment_03.csv', 'average_sentiment_04.csv', 'average_sentiment_05.csv', 'average_sentiment_06.csv', 'average_sentiment_07.csv', 'average_sentiment_08.csv', 'average_sentiment_09.csv', 'average_sentiment_10.csv', 'average_sentiment_11.csv', 'average_sentiment_12.csv', 'average_sentiment_13.csv', 'average_sentiment_14.csv', 'average_sentiment_15.csv', 'average_sentiment_16.csv', 'average_sentiment_17.csv', 'average_sentiment_18.csv', 'average_sentiment_19.csv', 'average_sentiment_20.csv', 'average_sentiment_21.csv', 'average_sentiment_22.csv', 'average_sentiment_23.csv', 'average_sentiment_24.csv', 'average_sentiment_25.csv', 'average_sentiment_26.csv', 'average_sentiment_27.csv', 'average_sentiment_28.csv', 'average_sentiment_29.csv', 'average_sentiment_30.csv']


The code here is taken from the Plotly library documentation and adapted to suit our needs
* https://plotly.com/python/choropleth-maps/
* https://plotly.com/python/mapbox-county-choropleth/
* https://plotly.com/python/sliders/
* https://plotly.com/python/reference/layout/updatemenus/
* https://plotly.com/python/reference/choropleth/
* https://plotly.com/python/colorscales/

We create a plotly figure of a chloropleth map for average sentiment with a slider to adjust the day

In [8]:

fig = go.Figure()

for step in np.arange(len(paths_list)):
    average_state_sentiment = pd.read_csv(paths_list[step], engine='python')

    fig.add_trace(
        go.Choropleth(
        locations=average_state_sentiment['state'], # Spatial coordinates
        z = average_state_sentiment['sentiment'].astype(float), # Data to be color-coded
        locationmode = 'USA-states', # set of locations match entries in `locations`
        colorscale = 'Blues',
        reversescale= True,
        colorbar_title = "Average Sentiment",
        text=average_state_sentiment.apply(lambda row: "Sentiment: "+f"{round(row['sentiment'],3)}" + ", Magnitude: " f"{round(row['magnitude'],3)}<br>{row['state']}", axis=1), # text to display on hover
        hoverinfo="text",
        visible=False,
        name= '{}'.format(step),
        zmax=0.00, #maximum color value
        zmin=-0.25 #minimum color value

    )
    )


#fig.update_traces(hovertemplate='<extra></extra>', selector=dict(type='histogram'))

fig.update_layout(
    geo_scope='usa', # limit map scope to USA
)

fig.update_layout(height=500, width=800, hovermode='x unified') #changes figure size

fig.update_layout(title_text = 'Average Sentiment on COVID Tweets in the US, April 2020')
fig.data[0].visible = True #sets day 1 to be visible when opening/loading the plot


steps = []
for i in range(len(fig.data)):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig.data)},
              {"title": "Average Sentiment on COVID Tweets in the US, {}/04/2020".format(str(i+1))}],
        label= "{}".format(i+1)
    )

    step["args"][0]["visible"][i] = True
  # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    active=10,
    currentvalue={"prefix": "Date: "},
    pad={"t": 50},
    steps=steps
)]
fig.update_layout(
    sliders=sliders
)


fig.show()



Saving the plot as html to embed on our website:

In [9]:
fig.write_html("chloropleth_full.html")
